In [1]:
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras.applications.vgg16 import VGG16



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [46]:
train_dir = ('/content/drive/MyDrive/dataset15s/train')
val_dir = ('/content/drive/MyDrive/dataset15s/val')
test_dir = ('/content/drive/MyDrive/dataset15s/test')

In [93]:
IMAGE_SIZE = 244
BATCH_SIZE = 16

train_datagen = ImageDataGenerator(
        rescale=1./255,
) 

valid_datagen = ImageDataGenerator(
        rescale=1./255,
       )


train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size =(IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    class_mode='categorical',
    color_mode='rgb'
)

valid_gen = valid_datagen.flow_from_directory(
    val_dir,
    target_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size= BATCH_SIZE,
    class_mode='categorical',
    color_mode= "rgb"
)
    


Found 711 images belonging to 3 classes.
Found 355 images belonging to 3 classes.


In [94]:
 base_model = tf.keras.applications.VGG16(input_shape=(224, 224, 3), include_top=False, weights="imagenet")


In [95]:
for layer in base_model.layers[:17]:
    layer.trainable = False

for i, layer in enumerate(base_model.layers):
    print(i, layer.name, layer.trainable)

0 input_9 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 False
16 block5_conv2 False
17 block5_conv3 True
18 block5_pool True


In [96]:
last_layer = base_model.get_layer('block5_pool')
last_output = last_layer.output
x = tf.keras.layers.GlobalMaxPooling2D()(last_output)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(3, activation='softmax')(x)



In [97]:
model = tf.keras.Model(base_model.input, x)


In [98]:
model.compile(optimizer='adam', loss=tf.keras.losses.categorical_crossentropy, metrics=['acc'])


In [ ]:
from matplotlib import pyplot as plt

history = model.fit(train_gen,epochs = 150,validation_data = valid_gen)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

Epoch 1/150
45/45 [==============================] - 12s 255ms/step - loss: 1.2960 - acc: 0.3741 - val_loss: 1.1024 - val_acc: 0.3042
Epoch 2/150
43/45 [===========================>..] - ETA: 0s - loss: 1.0730 - acc: 0.3976

In [ ]:
test_datagen = ImageDataGenerator(
        rescale=1./255)


test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size= BATCH_SIZE,
    class_mode='categorical'
)


pred = model.predict_generator(test_gen, steps=10, verbose=1)

Found 115 images belonging to 3 classes.


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


10/10 [==============================] - 31s 3s/step


In [ ]:
final_loss, final_acc = model.evaluate(valid_gen, verbose=0)
print("Final loss: {0:.6f}, final accuracy: {1:.6f}".format(final_loss, final_acc))

Final loss: 1.109834, final accuracy: 0.642202


In [ ]:
predicted_class = np.argmax(pred,axis=1)


In [ ]:
l = dict((v,k) for k,v in train_gen.class_indices.items())
prednames = [l[k] for k in predicted_class]


In [ ]:
filenames = test_gen.filenames


In [ ]:
import pandas as pd
finaldf = pd.DataFrame({'Filename': filenames,'Prediction': prednames})


In [ ]:
finaldf.to_csv("pred25s.csv")